<a href="https://colab.research.google.com/github/Julia891021/954/blob/0620/0607_5000_oldattention_%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

訓練

-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import shutil
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
import cv2
#文字前處理 jieba+去除標點符號
import string
!pip install jieba
import jieba
os.chdir('/content/gdrive/Shareddrives/954/文字/jieba')
jieba.set_dictionary('dict.txt')
import jieba.analyse
!pip install emoji
import emoji
#文字轉向量
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models.word2vec import Word2Vec
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
!pip install word2vec
import word2vec
import torch
from torch import nn
from torchvision import transforms
model = gensim.models.Word2Vec.load('/content/gdrive/Shareddrives/954/文字/word2vec_5000.model')
import torch
from torch import nn
import torch.nn.functional as tnf
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=882365c0773d66038044b530df429e4d665ea0374270e38920e80226afc5301b
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 557 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=156420 sha256=0f7b40830a88fcf8eee0653d8ed220cc3d0ce2e31622b88907aa55eb3ad274f9
  Stored in directory: 

In [4]:
class CNN(nn.Module):
    def __init__(self,dropout=0.5):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(       
            nn.Conv2d(
                in_channels=3,
                out_channels=16,       
                kernel_size=5,        
                stride=1,                  
                padding=2,                
            ),                          
            nn.ReLU(),                     
            nn.MaxPool2d(kernel_size=5),   
        )
        self.conv2 = nn.Sequential(       
            nn.Conv2d(16, 32, 5, 1, 2),  
            nn.ReLU(),                    
            nn.MaxPool2d(5),                
        )
        self.out = nn.Sequential(
            nn.Linear(1200,500),  # 輸入層與第一隱層結點數設定，全連線結構
            torch.nn.Sigmoid(),  # 第一隱層啟用函式採用sigmoid
            nn.Linear(500,100),  # 第一隱層與第二隱層結點數設定，全連線結構
            torch.nn.Sigmoid(),  # 第一隱層啟用函式採用sigmoid
            nn.Linear(100,50),  # 第二隱層與輸出層層結點數設定，全連線結構
            torch.nn.Sigmoid(), 
            nn.Linear(50,2),  
            nn.Softmax(dim=1) # 由於有兩個概率輸出，因此對其使用Softmax進行概率歸一化
        )
        self.dense = nn.Linear(12800,300)
        self.attention = nn.Linear(600,2)
        #model = gensim.models.Word2Vec.load('/content/gdrive/Shareddrives/954/文字/word2vec.model')
        weights = torch.FloatTensor(model.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights)
        self.embedding.requires_grad = False
        self.conv1_text = nn.Sequential(         # input shape (網紅個數,100,100)
            nn.Conv1d(
                in_channels=100,            # input height
                out_channels=32,            # n_filters
                kernel_size=7,              # filter size 
                stride=1,                   # filter movement/step 
                padding=3,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (網紅個數, 32, 100)
            nn.ReLU(),                      # activation
            nn.MaxPool1d(kernel_size=2),    # choose max value in 2, output shape (網紅個數,32, 50) , 100/2=50
        )
        self.flat = nn.Flatten()  #(網紅個數,32*50)
        self.dropout = nn.Dropout(p=dropout)  #(網紅個數,16*50)
        self.out_300 = nn.Linear(1600,300) #(16*50,300)
    def forward(self):
      #--------------------------------文字處理--------------------------------------------------------------
      #匯入官方網紅資料
      dataPath='/content/gdrive/Shareddrives/954/品牌資料'
      def loadExcel_official(dataPath):
        i=0
        foldername = os.listdir(dataPath)
        file_list = []
        #print(foldername)
        for folder in foldername:
          for filename in os.listdir(os.path.join(dataPath,folder)):
            if filename=='文字':
              for file in os.listdir(os.path.join(dataPath,folder,filename)):
                file_list.append(file)
                if i==0:
                  df=pd.read_excel(os.path.join(dataPath,folder,filename,file))
                  df_new=df
                  i+=1
                else:
                  df=pd.read_excel(os.path.join(dataPath,folder,filename,file))
                  df_new=pd.concat([df_new,df],axis=1,ignore_index=True)
                  i+=1
        return df_new, file_list, folder

      df_official, official_name, folder =loadExcel_official(dataPath)
      df_official = pd.concat([df_official[0], df_official[1], df_official[2]], ignore_index = True)
      df_official = pd.DataFrame(df_official)

      #匯入網紅資料
      dataPath='/content/gdrive/Shareddrives/954/網紅資料' 
      def loadExcel(dataPath, brand):
        df = pd.read_excel('/content/gdrive/Shareddrives/954/網紅資料/網紅_分類表.xlsx')
        celebrity = pd.concat([df['celebrity(folder_name)'], df['mouggan']], axis = 1)
        celebrity.set_index('celebrity(folder_name)',inplace=True)
        official = []
        for name in official_name:
          name = name.strip('.xlsx')
          official.append(name)
        celebrity = celebrity.drop(index = official)
        foldername = celebrity.index
        i=0
        j=0
        for folder in foldername:
          if celebrity[brand][folder]==1:
            for filename in os.listdir(os.path.join(dataPath,folder)):
              if filename=='文字':
                for file in os.listdir(os.path.join(dataPath,folder,filename)):
                  if i==0:
                    df_one=pd.read_excel(os.path.join(dataPath,folder,filename,file))
                    df_new_one=df_one
                    i+=1
                  else:
                    df_one=pd.read_excel(os.path.join(dataPath,folder,filename,file))
                    df_new_one=pd.concat([df_new_one,df_one],axis=1,ignore_index=True)
                    i+=1
          if celebrity[brand][folder]==0:
            for filename in os.listdir(os.path.join(dataPath,folder)):
              if filename=='文字':
                for file in os.listdir(os.path.join(dataPath,folder,filename)):
                  if j==0:
                    df_zero=pd.read_excel(os.path.join(dataPath,folder,filename,file))
                    df_new_zero=df_zero
                    j+=1
                  else:
                    df_zero=pd.read_excel(os.path.join(dataPath,folder,filename,file))
                    df_new_zero=pd.concat([df_new_zero,df_zero],axis=1,ignore_index=True)
                    j+=1
        return df_new_one,df_new_zero
      df_one,df_zero=loadExcel(dataPath,'mouggan')

      #文字前處理
      def text_preprocessing(df):
        #所有貼文合併成一篇
        def text_concate(df):
          result = []
          for j in range(df.shape[1]): 
              new1 = []
              for i in range(df.shape[0]):
                if df[j][i] != float('nan'):
                  new1.append(df[j][i])
              new2 = "".join(str(new1))
              result.append(new2)
          return result
        df_new = pd.DataFrame(text_concate(df), columns = ['text'])

        #emoji to text
        def emoji_to_text(df):
          #print(df['text'])
          for i in range(df_new.shape[0]):
            df['text'][i] = emoji.demojize(df['text'][i], delimiters=("",""))
          return df
        df_new = emoji_to_text(df_new)
        def remove_punctuation(df_new):
          for i in range(df_new.shape[0]):
            df_new['text'][i] = ''.join([i for i in df_new['text'][i] if i not in string.punctuation])
          return df_new
        df_new = remove_punctuation(df_new)
        #jieba斷句
        def jieba_text(df):
          for i in range(df.shape[0]):
            word = []
            word.append(list(jieba.cut(df['text'][i], cut_all=False))[:-1]) # 精確模式
            for j in range(len(word)):
              sentence = ' '.join(word[0])
            df['text'][i] = sentence
            #print(word[0])
          return df
        df_new = jieba_text(df_new)
        return df_new
      df_brand_zero_cut = text_preprocessing(df_zero)
      df_brand_one_cut = text_preprocessing(df_one)
      df_brand_official_cut= text_preprocessing(df_official)
      df_brand_zero_cut = df_brand_zero_cut.drop([14])
      df_brand_zero_cut = df_brand_zero_cut.reset_index()

      #word2vec轉向量
      #load word2vec pre-trained model
      def word_to_vector(df):
        def word_split(tokens, size):
            vec = np.zeros(size).reshape((1, size))
            count = 0
            for word in tokens:
                try:
                    vec += model[word].reshape((1, size))
                    count += 1.
                except KeyError:  # handling the case where the token is not in vocabulary
                    continue
            if count != 0:
                vec /= count
            return vec
        #轉成array的形式等等丟model
        tokenized_tweet = df['text'].apply(lambda x: x.split())
        # tokenized_tweet = tokenized_tweet.tolist()
        #wordvec_arrays = np.zeros((len(tokenized_tweet), 100))
        query_id = torch.tensor(np.zeros((len(tokenized_tweet),100)),dtype=torch.int64)
        for i in range(len(tokenized_tweet)):
          for j in range(100):
            if tokenized_tweet[i][j] in model.wv:
              query_id[i][j] = torch.tensor(model.wv.vocab[tokenized_tweet[i][j]].index)
            else:
              continue
        #w2v_df = torch.tensor(wordvec_arrays)
        return query_id

      official_input = word_to_vector(df_brand_official_cut)
      one_input = word_to_vector(df_brand_one_cut)
      zero_input = word_to_vector(df_brand_zero_cut)

      def cnn_text(x):       #torch.Size([網紅個數, 100])
        #print(x.shape)
        x = self.embedding(x)  #torch.Size([網紅個數, 100, 100])
        #print(x.shape)
        x = self.dropout(x)   #torch.Size([網紅個數, 100, 100])
        #print(x.shape)
        x = self.conv1_text(x)  #torch.Size([網紅個數, 32, 50])
        #print(x.shape)
        x = self.dropout(x)   #torch.Size([網紅個數, 32, 50])
        #print(x.shape)
        x = self.flat(x)    #torch.Size([網紅個數, 1600])
        #print(x.shape)
        x = self.dropout(x)  #torch.Size([網紅個數, 1600])
        #print(x.shape)
        x = self.out_300(x)   #torch.Size([網紅個數, 300])
        #print(x.shape)
        return x    # return x for visualization
      
      output_official_cnn = cnn_text(official_input)
      output_one_cnn = cnn_text(one_input)
      output_zero_cnn = cnn_text(zero_input)

      def concat_brand(tensor_other,tensor_brand):
        for i in range(tensor_other.shape[0]):
          if i==0:
            tensor_brand_new=tensor_brand
            i+=1
          else:
            tensor_brand_new=torch.cat((tensor_brand_new,tensor_brand), 0)
            i+=1
        output = torch.cat((tensor_other, tensor_brand_new), 1)
        return output
      #--------------------------------圖片處理--------------------------------------------------------------
      # 品牌
      os.chdir('/content/gdrive/Shareddrives/954/品牌資料/' + 'mouggan/'+'圖片')
      img_list = os.listdir()
      result = []
      for i in range (60) :
              if (img_list[i] != '.DS_Store'):
                  img = cv2.imread( img_list[i] )
                  img = cv2.resize( img, (500, 500))
                  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                  transform = transforms.Compose([
                      transforms.ToTensor()
                  ])
                  tensor = transform(img)
                  tensor = torch.reshape( tensor, ( 1,3,500,500) )
                  tensor = self.conv1(tensor)
                  tensor = self.conv2(tensor)
                  output = tensor.view(tensor.size(0), -1) 
                  output = self.dense(output)# !change(1,12800) to (1,300)
                  output = output.detach().numpy()
                  result.append(output)
      result= torch.tensor(result).float()
      result = torch.reshape(result, (result.shape[0], result.shape[2]))
      result = result.detach().numpy()
      mean = np.mean(result, axis=0) 
      result = []
      result.append(mean)
      brand_img_vector= torch.tensor(result).float()
      
      # 網紅
      os.chdir('/content/gdrive/Shareddrives/954/網紅資料/')
      excel = pd.read_excel('網紅_分類表.xlsx')
      folder = excel['celebrity(folder_name)']
      brand = excel['mouggan']

      folder_1 = []
      folder_0 = []
      for i in range(len(brand)):
        if brand[i] == 1:
          folder_1.append(i)
        else:
          folder_0.append(i)

      celebrity_img_0 = []
      celebrity_img_1 = []
      for i in range(len(folder_1)):
        celebrity_img_1.append(folder[folder_1[i]])
      for i in range(len(folder_0)):
        celebrity_img_0.append(folder[folder_0[i]])
      celebrity_img_1.remove('mouggan')
      celebrity_img_1.remove('manjumilin')
      celebrity_img_1.remove('egg204')

      celebrity_img_vector_1 = torch.tensor([])
      for folder_name in celebrity_img_1 :
        os.chdir('/content/gdrive/Shareddrives/954/網紅資料/' + folder_name + '/圖片' )
        img_list = os.listdir()
        result = []
        for i in range (20) :
            if (img_list[i] != '.DS_Store'):
                img = cv2.imread( img_list[i] )
                img = cv2.resize( img, (500, 500))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                transform = transforms.Compose([
                    transforms.ToTensor()
                ])
                tensor = transform(img)
                tensor = torch.reshape( tensor, ( 1,3,500,500) )
                tensor = self.conv1(tensor)
                tensor = self.conv2(tensor)
                output = tensor.view(tensor.size(0), -1) 
                output = self.dense(output)# !change(1,12800) to (1,300)
                output = output.detach().numpy()
                result.append(output)
        result= torch.tensor(result).float()
        result = torch.reshape( result, (result.shape[0], result.shape[2]))
        result = result.detach().numpy()
        mean = np.mean(result, axis=0) 
        result = []
        result.append(mean)
        result= torch.tensor(result).float()
        celebrity_img_vector_1 = torch.cat( ( celebrity_img_vector_1, result), 0)
      celebrity_1 = torch.cat((celebrity_img_vector_1, output_one_cnn), 1)  
      celebrity_1 = celebrity_1.repeat(2,1) 

      celebrity_img_vector_0 = torch.tensor([])
      for folder_name in celebrity_img_0 :
        os.chdir('/content/gdrive/Shareddrives/954/網紅資料/' + folder_name + '/圖片' )
        img_list = os.listdir()
        result = []
        for i in range (20) :
          if (img_list[i] != '.DS_Store'):
              img = cv2.imread( img_list[i] )
              img = cv2.resize( img, (500, 500))
              img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
              transform = transforms.Compose([
                  transforms.ToTensor()
              ])
              tensor = transform(img)
              tensor = torch.reshape( tensor, ( 1,3,500,500) )
              tensor = self.conv1(tensor)
              tensor = self.conv2(tensor)
              output = tensor.view(tensor.size(0), -1) 
              output = self.dense(output)# !change(1,12800) to (1,300)
              output = output.detach().numpy()
              result.append(output)
        result= torch.tensor(result).float()
        result = torch.reshape( result, (result.shape[0], result.shape[2]))
        result = result.detach().numpy()
        mean = np.mean(result, axis=0) 
        result = []
        result.append(mean)
        result= torch.tensor(result).float()
        celebrity_img_vector_0 = torch.cat( ( celebrity_img_vector_0, result), 0)
      celebrity_0 = torch.cat((celebrity_img_vector_0, output_zero_cnn), 1)

      brand_img_vector = brand_img_vector.repeat(366,1)
      output_official_cnn = output_official_cnn.repeat(366,1)
      brand_all = torch.cat((brand_img_vector, output_official_cnn), 1) 
      celebrity_all = torch.cat((celebrity_0, celebrity_1), 0) 

      # attention 標準注意力
      for i in range(celebrity_all.shape[0]):
        image = celebrity_all[i][:300].reshape(1,300)
        text = celebrity_all[i][300:].reshape(1,300)
        all = torch.cat((image,text),0)
        all = all.detach().numpy()
        all_weight = np.matmul(all,all.T)
        all_weight = torch.tensor(all_weight).float()
        all_weight=tnf.softmax(all_weight,dim=1)
        all_weight=all_weight.detach().numpy()
        all_new=np.matmul(all_weight,all)
        all_new = torch.tensor(all_new).float()
        celebrity_all[i] = all_new.reshape(600)

      for i in range(brand_all.shape[0]):
        image = brand_all[i][:300].reshape(1,300)
        text = brand_all[i][300:].reshape(1,300)
        all = torch.cat((image,text),0)
        all = all.detach().numpy()
        all_weight = np.matmul(all,all.T)
        all_weight = torch.tensor(all_weight).float()
        all_weight=tnf.softmax(all_weight,dim=1)
        all_weight=all_weight.detach().numpy()
        all_new=np.matmul(all_weight,all)
        all_new = torch.tensor(all_new).float()
        brand_all[i] = all_new.reshape(600)

      # # attention 神經網路
      # for i in range(celebrity_all.shape[0]):
      #   image = celebrity_all[i][:300].reshape(1,300)
      #   text = celebrity_all[i][300:].reshape(1,300)
      #   all = celebrity_all[i]
      #   all_weight = self.attention(all).reshape(1,2)
      #   all_weight = tnf.softmax(all_weight,dim=1)
      #   text_weight = all_weight[0][0].reshape(1,1).float()
      #   image_weight = all_weight[0][1].reshape(1,1).float()
      #   weighted_text = torch.mm(text_weight,text)
      #   weighted_image = torch.mm(image_weight,image)
      #   all_new = torch.cat((weighted_text,weighted_image),1)
      #   celebrity_all[i] = all_new.reshape(600)

      # for i in range(brand_all.shape[0]):
      #   image = brand_all[i][:300].reshape(1,300)
      #   text = brand_all[i][300:].reshape(1,300)
      #   all = brand_all[i]
      #   all_weight = self.attention(all).reshape(1,2)
      #   all_weight = tnf.softmax(all_weight,dim=1)
      #   text_weight = all_weight[0][0].reshape(1,1).float()
      #   image_weight = all_weight[0][1].reshape(1,1).float()
      #   weighted_text = torch.mm(text_weight,text)
      #   weighted_image = torch.mm(image_weight,image)
      #   all_new = torch.cat((weighted_text,weighted_image),1)
      #   brand_all[i] = all_new.reshape(600)

      print(celebrity_all.shape)
      print(brand_all.shape)
      x_t = torch.cat((celebrity_all, brand_all), 1) 
      output = self.out(x_t)

      return output  

In [5]:
cnn = CNN()
cnn

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Sequential(
    (0): Linear(in_features=1200, out_features=500, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=500, out_features=100, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=50, out_features=2, bias=True)
    (7): Softmax(dim=1)
  )
  (dense): Linear(in_features=12800, out_features=300, bias=True)
  (attention): Linear(in_features=600, out_features=2, bias=True)
  (embedding): Embedding(4237114, 100)
  (conv1_text): Sequential(
    (0): Conv1d(100, 32, kernel_size=(7,),

In [ ]:
optimizer = torch.optim.SGD(cnn.parameters(),lr=0.01) # 優化器使用隨機梯度下降，傳入網路引數和學習率
loss_func = torch.nn.CrossEntropyLoss() # 損失函式使用交叉熵損失函式

y_1_t = torch.ones(174) # Y 9+78=87 87*2=174
y_0_t = torch.zeros(192) # Y

y_t = torch.cat((y_0_t,y_1_t),0)
# 模型訓練
num_epoch = 150 # 最大迭代更新次數
for epoch in range(num_epoch):
  y_p = cnn()  # 喂資料並前向傳播
  loss = loss_func(y_p,y_t.long()) # 計算損失
  optimizer.zero_grad()  # 清除梯度
  loss.backward()  # 計算梯度，誤差回傳
  optimizer.step()  # 根據計算的梯度，更新網路中的引數
    # if epoch % 10 == 0:
    #     print('epoch: {}, loss: {}'.format(epoch, loss.data.item()))
  print(loss.data.item())

Building prefix dict from /content/gdrive/Shareddrives/954/文字/jieba/dict.txt ...
Dumping model to file cache /tmp/jieba.u991154c518f2a731fe75d5ff0d98fecd.cache
Loading model cost 0.773 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:251: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)


torch.Size([366, 600])
torch.Size([366, 600])
0.6925977468490601
torch.Size([366, 600])
torch.Size([366, 600])
0.6925320029258728
torch.Size([366, 600])
torch.Size([366, 600])
0.692625105381012
torch.Size([366, 600])
torch.Size([366, 600])
0.6924645304679871
torch.Size([366, 600])
torch.Size([366, 600])
0.6924841403961182
torch.Size([366, 600])
torch.Size([366, 600])
0.6924542188644409
torch.Size([366, 600])
torch.Size([366, 600])
0.6924432516098022
torch.Size([366, 600])
torch.Size([366, 600])
0.6924879550933838
torch.Size([366, 600])
torch.Size([366, 600])
0.6924780607223511
torch.Size([366, 600])
torch.Size([366, 600])
0.6924229264259338
torch.Size([366, 600])
torch.Size([366, 600])
0.6923584938049316
torch.Size([366, 600])
torch.Size([366, 600])
0.6923707723617554
torch.Size([366, 600])
torch.Size([366, 600])
0.6923573017120361
torch.Size([366, 600])
torch.Size([366, 600])
0.6923081278800964
torch.Size([366, 600])
torch.Size([366, 600])
0.6923253536224365
torch.Size([366, 600])
tor

In [ ]:
print("所有樣本的預測標籤: \n",torch.max(y_p,dim = 1)[1])

attention 測試

In [ ]:
# 神經網路注意力
celebrity_all = torch.rand(201,600)
brand_all = torch.rand(201,600)

# attention 神經網路
for i in range(celebrity_all.shape[0]):
  image = celebrity_all[i][:300].reshape(1,300)
  text = celebrity_all[i][300:].reshape(1,300)
  all = celebrity_all[i]
  all_weight = self.attention(all)
  all_weight = tnf.softmax(all_weight,dim=1)
  text_weight = all_weight[0][0].reshape(1,1).float()
  image_weight = all_weight[0][1].reshape(1,1).float()
  weighted_text = torch.mm(text_weight,text)
  weighted_image = torch.mm(image_weight,image)
  all_new = torch.cat((weighted_text,weighted_image),1)
  celebrity_all[i] = all_new.reshape(600)

for i in range(brand_all.shape[0]):
  image = brand_all[i][:300].reshape(1,300)
  text = brand_all[i][300:].reshape(1,300)
  all = brand_all[i]
  all_weight = self.attention(all)
  all_weight = tnf.softmax(all_weight,dim=1)
  text_weight = all_weight[0][0].reshape(1,1).float()
  image_weight = all_weight[0][1].reshape(1,1).float()
  weighted_text = torch.mm(text_weight,text)
  weighted_image = torch.mm(image_weight,image)
  all_new = torch.cat((weighted_text,weighted_image),1)
  brand_all[i] = all_new.reshape(600)